In [ ]:
import requests
import csv
import re
from bs4 import BeautifulSoup
import pyperclip
import time
from urllib.parse import urljoin

# Base URL to scrape
BASE_URL = "https://careers.singhealth.com.sg/search/?createNewAlert=false&q=&optionsFacetsDD_customfield3="
HEADERS = {"User-Agent": "Mozilla/5.0"}
DELAY = 1  # seconds between requests 

def fetch_jobs(startrow=0, max_pages=23):
    """Scrape job postings from the SingHealth careers page with pagination support."""
    jobs = []
    seen_links = set()
    page_count = 0
    
    while True:
        # Build URL with current pagination parameter
        url = f"{BASE_URL}&startrow={startrow}"
        print(f"Fetching page {page_count + 1} (startrow={startrow})...")
        
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            if response.status_code != 200:
                print(f"Failed to retrieve the webpage (status code: {response.status_code})")
                break
                
            soup = BeautifulSoup(response.text, 'html.parser')
            job_listings = soup.find_all('div', class_='job-listing') or soup.find_all('li', class_='job-item')
            
            # If the above selectors don't work, fall back to a more general approach
            if not job_listings:
                job_listings = soup.find_all('a', href=True)
            
            current_page_jobs = 0
            
            for job in job_listings:
                job_link = job if job.name == 'a' else job.find('a', href=True)
                
                if not job_link or 'href' not in job_link.attrs or "job" not in job_link['href'].lower():
                    continue
                    
                link = urljoin("https://careers.singhealth.com.sg", job_link['href'])
                
                if link in seen_links:
                    continue
                    
                seen_links.add(link)
                title = job_link.text.strip()
                
                # Extract hospital name (if available)
                hospital = "Unknown"
                match = re.search(r'\((.*?)\)', title)
                if match:
                    hospital = match.group(1)
                    # Remove hospital from title if found in parentheses
                    title = title.replace(f"({hospital})", "").strip()
                
                jobs.append({
                    "Job Title": title, 
                    "Hospital": hospital, 
                    "Link": link,
                    "Excel Link": f'=HYPERLINK("{link}", "View Job")'  # Excel formula for clickable link
                })
                current_page_jobs += 1
            
            print(f"Found {current_page_jobs} jobs on this page")
            
            # Check if we should continue to next page
            if current_page_jobs == 0:
                print("No more jobs found, stopping pagination.")
                break
                
            # Increment for next page (assuming 25 items per page - adjust if needed)
            startrow += 25
            page_count += 1
            
            # Stop if we've reached the maximum requested pages
            if max_pages is not None and page_count >= max_pages:
                print(f"Reached maximum page limit of {max_pages}.")
                break
                
            # Be polite and don't overwhelm the server
            time.sleep(DELAY)
            
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            break
            
    return jobs

def copy_jobs_to_clipboard(jobs):
    """Copy job listings in Excel-friendly format to clipboard."""
    if not jobs:
        print("No jobs found to copy.")
        return
        
    # Create Excel-friendly TSV (tab-separated) format
    excel_data = "Job Title\tHospital\tLink\tExcel Link\n"
    
    for job in jobs:
        # Escape special characters and ensure proper formatting
        title = job['Job Title'].replace('"', '""')
        hospital = job['Hospital'].replace('"', '""')
        excel_data += f'"{title}"\t"{hospital}"\t"{job["Link"]}"\t{job["Excel Link"]}\n'
    
    pyperclip.copy(excel_data)
    print(f"✅ {len(jobs)} job listings copied to clipboard in Excel-friendly format!")
    print("Paste into Excel and the data will automatically appear in columns with clickable links.")

def save_jobs_to_excel(jobs, filename="singhealth_jobs.csv"):
    """Save job listings to a CSV file with Excel-friendly formatting."""
    if not jobs:
        print("No jobs found to save.")
        return
        
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Job Title", "Hospital", "Link", "Excel Link"])
        writer.writeheader()
        
        for job in jobs:
            # Clean up data for CSV output
            cleaned_job = {
                "Job Title": job['Job Title'],
                "Hospital": job['Hospital'],
                "Link": job['Link'],
                "Excel Link": job['Excel Link']
            }
            writer.writerow(cleaned_job)
    
    print(f"✅ {len(jobs)} job listings saved to {filename}")
    print("Open in Excel and you'll see:")
    print("- Proper columns for Job Title, Hospital, and Link")
    print("- Clickable hyperlinks in the 'Excel Link' column")

def save_jobs_to_csv(jobs, filename="singhealth_jobs.csv"):
    """Save job listings to a plain CSV file."""
    if not jobs:
        print("No jobs found to save.")
        return
        
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Job Title", "Hospital", "Link"])
        writer.writeheader()
        
        for job in jobs:
            writer.writerow({
                "Job Title": job['Job Title'],
                "Hospital": job['Hospital'],
                "Link": job['Link']
            })
    
    print(f"✅ {len(jobs)} job listings saved to {filename} (plain CSV)")

# Fetch jobs with pagination
# Set max_pages=None to scrape all available pages, or set to a specific number
jobs = fetch_jobs(startrow=0, max_pages=23)  # Example: scrape first 3 pages

if jobs:
    copy_jobs_to_clipboard(jobs)
    save_jobs_to_excel(jobs)
else:
    print("No jobs found.")

Fetching page 1 (startrow=0)...
Found 25 jobs on this page
Fetching page 2 (startrow=25)...
Found 25 jobs on this page
Fetching page 3 (startrow=50)...
Found 25 jobs on this page
Fetching page 4 (startrow=75)...
Found 25 jobs on this page
Fetching page 5 (startrow=100)...
Found 25 jobs on this page
Fetching page 6 (startrow=125)...
Found 25 jobs on this page
Fetching page 7 (startrow=150)...
Found 25 jobs on this page
Fetching page 8 (startrow=175)...
Found 25 jobs on this page
Fetching page 9 (startrow=200)...
Found 25 jobs on this page
Fetching page 10 (startrow=225)...
Found 25 jobs on this page
Fetching page 11 (startrow=250)...
Found 25 jobs on this page
Fetching page 12 (startrow=275)...
Found 25 jobs on this page
Fetching page 13 (startrow=300)...
Found 25 jobs on this page
Fetching page 14 (startrow=325)...
Found 25 jobs on this page
Fetching page 15 (startrow=350)...
Found 25 jobs on this page
Fetching page 16 (startrow=375)...
Found 25 jobs on this page
Fetching page 17 (star